In [ ]:
#Импорт необходимых для работы библиотек

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split

#Подготавливаем тестовые данные
train = pd.read_csv('../input/digit-recognizer/train.csv')
test = pd.read_csv('../input/digit-recognizer/test.csv')

X_train = (train.drop('label', axis=1) / 255).values.reshape(-1, 28, 28, 1)
X_test = (test / 255).values.reshape(-1, 28, 28, 1)
y_train = np.array(pd.get_dummies(train['label']))


datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1
)

inputs = tf.keras.Input(shape=(28, 28, 1))

x = tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu')(inputs)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=5, strides=2, padding='same', activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.4)(x)

x = tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=5, strides=2, padding='same', activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.4)(x)

x = tf.keras.layers.Conv2D(filters=128, kernel_size=4, activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dropout(0.4)(x)
outputs = tf.keras.layers.Dense(10, activation='softmax')(x)

#Создаем модель
model = tf.keras.Model(inputs=inputs, outputs=outputs)


X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, train_size=0.9, shuffle=True, random_state=0
)

#Тренируем модель
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']   
)

history = model.fit_generator(
    datagen.flow(X_train, y_train, batch_size=64),
    validation_data=(X_val, y_val),
    epochs=20,
    callbacks=[
        tf.keras.callbacks.LearningRateScheduler(
            lambda x: 1e-3 * 0.95 ** x, verbose=0
        )
    ]
)

In [ ]:
predictions = model.predict(X_test).argmax(axis=1)
sample_submission = pd.read_csv('../input/digit-recognizer/sample_submission.csv')
sample_submission['Label'] = predictions
sample_submission.to_csv('submission.csv', index=False)